# AMBRS demo — **CAMP** H₂SO₄ condensation (MAM4 & PartMC, side - side via PPE)

This notebook does a **minimal** CAMP - based setup using the **AMBRs PPE generator** on the `analysis_clean` branch:

- Creates two runs via PPE: **MAM4** and **PartMC**
- Writes a **CAMP `namelist`** into each run directory (so both models use CAMP chemistry)
- Runs both models
- Plots gas - phase **H₂SO₄** and a heuristic sum of **condensed sulfate**

> ⚠️ Assumptions:
> - You are running this notebook inside the **AMBRS repo** (branch `analysis_clean`) or in an environment where the `ambrs` package is importable.
> - You have already run `pip install -r requirements` to install `ambrs` dependencies
> - `mam4` and `partmc` executables are already installed and discoverable on your `PATH`.
> - NetCDF/HDF5/CAMP runtime libs are on the loader path (see the Environment cell).

---


In [1]:
from pathlib import Path
import os, sys, shutil, subprocess, platform

# --- Config ---
RUN_ROOT = Path("runs/camp_h2so4_demo").resolve()  # change if desired
RUN_ROOT.mkdir(parents=True, exist_ok=True)

# CAMP keys (override by setting env before running a cell)
CAMP_CONFIG_KEY = os.environ.get("CAMP_CONFIG_KEY", "BOX_MODEL")
CAMP_MECH_KEY   = os.environ.get("CAMP_MECH_KEY",   "H2SO4_COND")

print("Repo cwd:", Path.cwd())
print("Run root:", RUN_ROOT)
print("CAMP config key:", CAMP_CONFIG_KEY)
print("CAMP mech key:", CAMP_MECH_KEY)

# --- Loader path hint (macOS) ---
if sys.platform == "darwin":
    conda = os.environ.get("CONDA_PREFIX", "")
    if conda:
        dy = os.environ.get("DYLD_LIBRARY_PATH", "")
        if (conda + "/lib") not in (dy or ""):
            print("\ud83d\udca1 macOS hint: you may need to run:")
            print(f'export DYLD_LIBRARY_PATH="{conda}/lib:$DYLD_LIBRARY_PATH"')
    else:
        print("Note: CONDA_PREFIX not set. Make sure HDF5/NetCDF/CAMP libs are on DYLD_LIBRARY_PATH.")

# --- Check ambrs import & version ---
try:
    import ambrs
    print("ambrs package:", getattr(ambrs, "__file__", "<unknown file>"))
except Exception as e:
    print("ERROR: Could not import 'ambrs'. Ensure you're on the AMBRS repo 'analysis_clean' or have it installed.")
    raise

# --- Check executables ---
for exe in ["mam4", "partmc"]:
    p = shutil.which(exe)
    print(f"{exe}: {p}")
    if p is None:
        raise SystemExit(f"'{exe}' not found on PATH.")


ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x105923880>>)
Traceback (most recent call last):
  File "/Users/fier887/miniforge3/envs/ambrs_camp_sync/lib/python3.12/site-packages/jupyter_client/session.py", line 100, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 248-249: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/fier887/miniforge3/envs/ambrs_camp_sync/lib/python3.12/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/Users/fier887/miniforge3/envs/ambrs_camp_sync/lib/python3.12/site-packages/ipykernel/iostream.py", line 656, in _flush
    self.session.send(
  File "/Users/fier887/miniforge3/envs/ambrs_cam

ambrs package: /Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/ambrs/__init__.py
mam4: /Users/fier887/miniforge3/envs/ambrs_camp_sync/bin/mam4
partmc: /Users/fier887/miniforge3/envs/ambrs_camp_sync/bin/partmc


## 1) Write PPE configuration (two scenarios: `mam4` and `partmc`)

We keep this minimal and generic; if your PPE schema differs slightly, edit the YAML below.


In [2]:
ppe_yaml = f"""
meta:
  experiment: "camp_h2so4_demo"

defaults:
  env:
    temperature_K: 298.15
    pressure_Pa: 101325.0
    RH: 0.50
    dt_s: 1.0
    duration_s: 600.0
  gas:
    SO2:   0.0
    H2SO4: 1.0e-12
  aero:
    modes:   [aitken, accumulation, coarse, primary_carbon]
    qso4:    [1e-12, 1e-12, 1e-12, 0.0]
    qpom:    [0.0,   0.0,   0.0,   0.0]
    qsoa:    [0.0,   0.0,   0.0,   0.0]
    qbc:     [0.0,   0.0,   0.0,   0.0]
    qdst:    [0.0,   0.0,   0.0,   0.0]
    qncl:    [0.0,   0.0,   0.0,   0.0]
    qmom:    [0.0,   0.0,   0.0,   0.0]
    qaerwat: [0.0,   0.0,   0.0,   0.0]
    GMD:     [2.0e-8, 1.0e-7, 5.0e-6, 5.0e-8]
    GSD:     [1.6, 1.6, 1.6, 1.6]
    numc:    [2.0e8, 1.0e8, 1.0e6, 1.0e7]

scenarios:
  - id: mam4
    model: mam4
  - id: partmc
    model: partmc
"""

config_path = RUN_ROOT / "ppe_config.yaml"
config_path.write_text(ppe_yaml)
print("Wrote:", config_path)
print(config_path.read_text())


Wrote: /Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs/camp_h2so4_demo/ppe_config.yaml

meta:
  experiment: "camp_h2so4_demo"

defaults:
  env:
    temperature_K: 298.15
    pressure_Pa: 101325.0
    RH: 0.50
    dt_s: 1.0
    duration_s: 600.0
  gas:
    SO2:   0.0
    H2SO4: 1.0e-12
  aero:
    modes:   [aitken, accumulation, coarse, primary_carbon]
    qso4:    [1e-12, 1e-12, 1e-12, 0.0]
    qpom:    [0.0,   0.0,   0.0,   0.0]
    qsoa:    [0.0,   0.0,   0.0,   0.0]
    qbc:     [0.0,   0.0,   0.0,   0.0]
    qdst:    [0.0,   0.0,   0.0,   0.0]
    qncl:    [0.0,   0.0,   0.0,   0.0]
    qmom:    [0.0,   0.0,   0.0,   0.0]
    qaerwat: [0.0,   0.0,   0.0,   0.0]
    GMD:     [2.0e-8, 1.0e-7, 5.0e-6, 5.0e-8]
    GSD:     [1.6, 1.6, 1.6, 1.6]
    numc:    [2.0e8, 1.0e8, 1.0e6, 1.0e7]

scenarios:
  - id: mam4
    model: mam4
  - id: partmc
    model: partmc



## 2) Generate the run directories via **AMBRs PPE**

This calls the PPE generator module to create `runs/camp_h2so4_demo/mam4` and `runs/camp_h2so4_demo/partmc` with inputs.


In [3]:
import subprocess, sys
cmd = [sys.executable, "-m", "ambrs.ppe.generate",
       "--config", str(config_path),
       "--out", str(RUN_ROOT)]
print("Running:", " ".join(cmd))
subprocess.check_call(cmd)
print("PPE generation complete.")
print("Subdirs:", [p.name for p in RUN_ROOT.iterdir() if p.is_dir()])


Running: /Users/fier887/miniforge3/envs/ambrs_camp_sync/bin/python -m ambrs.ppe.generate --config /Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs/camp_h2so4_demo/ppe_config.yaml --out /Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs/camp_h2so4_demo


/Users/fier887/miniforge3/envs/ambrs_camp_sync/bin/python: Error while finding module specification for 'ambrs.ppe.generate' (ModuleNotFoundError: __path__ attribute not found on 'ambrs.ppe' while trying to find 'ambrs.ppe.generate')


CalledProcessError: Command '['/Users/fier887/miniforge3/envs/ambrs_camp_sync/bin/python', '-m', 'ambrs.ppe.generate', '--config', '/Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs/camp_h2so4_demo/ppe_config.yaml', '--out', '/Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs/camp_h2so4_demo']' returned non-zero exit status 1.

## 3) Write **CAMP `namelist`** into each run directory

Both models will read `./namelist` for CAMP configuration.


In [ ]:
namelist_text = f"""&camp_config
  config_key = '{CAMP_CONFIG_KEY}'
/
&camp_mech
  mech_key   = '{CAMP_MECH_KEY}'
/
"""

for R in ["mam4", "partmc"]:
    rd = RUN_ROOT / R
    if not rd.exists():
        raise SystemExit(f"Expected run dir not found: {rd}")
    (rd / "namelist").write_text(namelist_text)
    print("Wrote namelist to:", rd / "namelist")

print("\nNamelist contents:\n", namelist_text)


## 4) Run **MAM4** and **PartMC**

If you run into shared-library errors, ensure NetCDF/HDF5/CAMP libs are on your loader path (see first cell).


In [ ]:
import subprocess

def run_model(exe, rundir):
    print(f"=== Running {exe} in", rundir)
    proc = subprocess.run([exe], cwd=rundir, capture_output=True, text=True)
    print(proc.stdout)
    if proc.returncode != 0:
        print(proc.stderr)
        raise SystemExit(f"{exe} failed with code {proc.returncode}")

run_model("mam4", RUN_ROOT/"mam4")
run_model("partmc", RUN_ROOT/"partmc")
print("Both models completed.")


## 5) Plot comparisons

We plot:
- gas-phase H₂SO₄ time series
- a heuristic sum over sulfate-like condensed variables


In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date
from pathlib import Path

def newest_nc(path: Path):
    files = sorted(path.glob("*.nc"), key=lambda p: p.stat().st_mtime)
    return files[-1] if files else None

def find_time_var(nc):
    for c in ("time","Time","TIME","t"):
        if c in nc.variables: return c
    for name, var in nc.variables.items():
        try:
            if getattr(var, "ndim", 0) == 1:
                return name
        except Exception:
            pass
    return None

def read_time(nc, tname):
    tv = np.asarray(nc.variables[tname][:])
    units = getattr(nc.variables[tname], "units", "")
    calendar = getattr(nc.variables[tname], "calendar", "standard")
    try:
        tt = num2date(tv, units=units, calendar=calendar)
        t0 = tt[0]
        return np.array([(x - t0).total_seconds() for x in tt], dtype=float)
    except Exception:
        return tv.astype(float)

def find_var(nc, patterns):
    for pat in patterns:
        rx = re.compile(pat, re.IGNORECASE)
        for name in nc.variables:
            if rx.search(name):
                return name
    return None

def gas_h2so4(nc, tname):
    pats = [r"^h2so4(_gas)?$", r"^gas_?h2so4$", r"vmr_?h2so4", r"mmr_?h2so4", r"conc.*h2so4", r"h2so4"]
    v = find_var(nc, pats)
    if not v: return None, None
    t = read_time(nc, tname) if tname else np.arange(nc.variables[v].shape[0], dtype=float)
    y = np.asarray(nc.variables[v][:], dtype=float)
    while y.ndim > 1:
        y = y.sum(axis=-1)
    return t, y

def sum_like_sulfate(nc, tname):
    t = read_time(nc, tname) if tname else None
    pats = [r"(^|_)so4(_a\\d)?$", r"sulfate.*mass", r"mass_.*so4", r"so4_.*mass", r"sulfate"]
    total = None; found = False
    for name, var in nc.variables.items():
        if name.lower() == (tname or "").lower(): continue
        if any(re.search(p, name, re.IGNORECASE) for p in pats):
            arr = np.asarray(var[:], dtype=float)
            if arr.ndim < 1: continue
            if t is not None and arr.shape[0] != len(t):
                continue
            while arr.ndim > 1:
                arr = arr.sum(axis=-1)
            total = arr if total is None else total + arr
            found = True
    return (t, total) if found else (t, None)

def load_series(nc_path: Path):
    with Dataset(str(nc_path)) as nc:
        tname = find_time_var(nc)
        tg, yg = gas_h2so4(nc, tname)
        tc, yc = sum_like_sulfate(nc, tname)
    return dict(tg=tg, yg=yg, tc=tc, yc=yc)

mam4_nc = newest_nc(RUN_ROOT/"mam4")
partmc_nc = newest_nc(RUN_ROOT/"partmc")
if mam4_nc is None or partmc_nc is None:
    raise SystemExit("Missing .nc outputs; check previous cell logs.")

M = load_series(mam4_nc)
P = load_series(partmc_nc)

# Gas H2SO4
fig = plt.figure()
ax = fig.add_subplot(111)
if M["yg"] is not None: ax.plot(M["tg"], M["yg"], label="MAM4")
if P["yg"] is not None: ax.plot(P["tg"], P["yg"], label="PartMC")
ax.set_xlabel("time [s]"); ax.set_ylabel("H2SO4 (file units)"); ax.set_title("Gas-phase H2SO4")
ax.legend(); fig.tight_layout()
gas_png = Path(RUN_ROOT) / "h2so4_gas_timeseries.png"; fig.savefig(gas_png, dpi=150); plt.close(fig)
print("Wrote", gas_png)

# Condensed sulfate (heuristic)
if M["yc"] is None and P["yc"] is None:
    print("No condensed sulfate-like variables found; skipping condensed plot.")
else:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    if M["yc"] is not None: ax.plot(M["tc"], M["yc"], label="MAM4 (sum sulfate-like)")
    if P["yc"] is not None: ax.plot(P["tc"], P["yc"], label="PartMC (sum sulfate-like)")
    ax.set_xlabel("time [s]"); ax.set_ylabel("sulfate condensed (sum)"); ax.set_title("Condensed sulfate")
    ax.legend(); fig.tight_layout()
    cond_png = Path(RUN_ROOT) / "sulfate_condensed_timeseries.png"; fig.savefig(cond_png, dpi=150); plt.close(fig)
    print("Wrote", cond_png)


---

## Troubleshooting

- **Missing symbols (HDF5/NetCDF)** on macOS:
  ```bash
  export DYLD_LIBRARY_PATH="$CONDA_PREFIX/lib:$DYLD_LIBRARY_PATH"
  ```
- **CAMP namelist keys rejected**: try
  ```bash
  export CAMP_MECH_KEY=CB05CL_AE5
  export CAMP_CONFIG_KEY=DEFAULT
  ```
  then rerun the *Write CAMP namelist* cell and the *Run models* cell.
- **PPE entrypoint**: if your PPE generator lives at a different module path,
  edit the command in the PPE cell:
  ```python
  cmd = [sys.executable, "-m", "ambrs.ppe.generate", "--config", str(config_path), "--out", str(RUN_ROOT)]
  ```
